In [1168]:
%matplotlib widget

import os
import csv
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import numpy.linalg as la
from sklearn.cluster import KMeans
from sklearn import svm, metrics

from skimage import io
from matplotlib.gridspec import GridSpec

# TeX typesetting
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('text', usetex=True)

# Aesthetics
plt.close('all')
plt.style.use('seaborn-pastel')

# Helper functions

In [1169]:
flatten = lambda l: np.array([item for sl in l for item in sl])
vectorize_im = lambda im,m: flatten(im.reshape((m,1)))

def bmatrix(a):
    """Returns a LaTeX bmatrix
    :a: numpy array
    :returns: LaTeX bmatrix as a string
    """
    if len(a.shape) > 2:
        raise ValueError('bmatrix can at most display two dimensions')
    lines = str(a).replace('[', '').replace(']', '').splitlines()
    rv = [r'\begin{bmatrix}']
    rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
    rv +=  [r'\end{bmatrix}']
    return '\n'.join(rv)

In [1170]:
def read_sift_descr(fname):
    descr = []
    with open(fname) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=" ")
        for row in csv_reader:
            descr.append(row)
    return np.array(descr, dtype="float32")

In [1171]:
def norm_im(v):
    mmin, mmax = np.min(v), np.max(v)
    im_v = v.reshape((h, w))
    output = np.zeros((h,w), dtype="float")
    fac = 255 / (mmax - mmin)
    for i in range(h):
        for j in range(w):
            output[i][j] = fac * (im_v[i][j] - mmin)
    return output.astype(np.uint8)

In [1172]:
def build_dataset(folder):
    classes = None
    label_dict = None
    train_descr = None
    test_descr = None
    
    for dirpaths, dirs, fns in os.walk(folder):
        if dirpaths == folder:
            # Still in the top dir
            n = len(dirs)
            classes = sorted(dirs, key=lambda x: str.lower(x))
            label_dict = {classes[i]:i for i in range(n)}
            train_descr = [[] for i in range(n)]
            test_descr  = [[] for i in range(n)]
        if "train" in dirpaths:
            cl = dirpaths.split('/')[-2]
            cl_id = label_dict[cl]
            descrs = []
            for f in sorted(fns, key=lambda x:str.lower(x)):
                if "_descr" in f:
                    descr = read_sift_descr("{}/{}".format(dirpaths, f))
                    descrs.append(descr)
            train_descr[cl_id] = descrs

        elif "test" in dirpaths:
            cl = dirpaths.split('/')[-2]
            cl_id = label_dict[cl]
            descrs = []
            for f in sorted(fns, key=lambda x:str.lower(x)):
                if "_descr" in f:
                    descr = read_sift_descr("{}/{}".format(dirpaths, f))
                    descrs.append(descr)
            test_descr[cl_id] = descrs
            
    return classes, label_dict, train_descr, test_descr

In [1173]:
def build_bow_hists(kmeans_inst, descrs):
    n = len(descrs)
    bow_hists = [[] for k in range(n)]
    labels = [[] for k in range(n)]
    for i in range(n):
        m = len(descrs[i])
        word_hist = [] 
        label = []
        for j in range(m):
            word = kmeans_inst.predict(descrs[i][j])
            hist, _ =  np.histogram(word,
                                    bins=range(kmeans_inst.n_clusters+1),
                                    density=True)
            label.append(i)
            word_hist.append(hist)
        bow_hists[i] = word_hist
        labels[i] = label
    return  np.array(bow_hists), np.array(labels)

# Q1

In [1174]:
# global vars
m = 0 # will modified, m = pq
n = 40 # training set length
test_sets = 2

### (a)

In [1175]:
training_f = []
for i in range(1, n+1):
    im = io.imread("../assets/assignment-5/problem1_files/faces_training/{}.pgm".format(i))
    h, w = np.shape(im)
    m = w * h
    training_f.append(vectorize_im(im, m))
    
training_f = np.array(training_f)
a = np.mean(training_f, axis=0)
X = np.empty((m, n))

for i in range(0, n):
    X[:,i] = training_f[i] - a
X *= np.sqrt(1/2)

U,S,VT = la.svd(X, full_matrices=True)

In [1176]:
t = 2000 # threshold
alpha = len(S[S > t])
U_alpha = U[:, :alpha]
training_y = []
for j in range(len(training_f)):
    y = U_alpha.T @ (training_f[j] - a)
    training_y.append(y)

In [1177]:
fig, ax = plt.subplots()
ax.set_axis_off()
eig_face = norm_im(a)
ax.imshow(eig_face, cmap=plt.cm.gray)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [1178]:
fig, ax = plt.subplots(1,4)
plt.subplots_adjust(wspace=0.1, hspace=0.0)
for i in range(4):
    ax[i].set_axis_off()
    eig_face = norm_im(U_alpha[:, i])
    ax[i].imshow(eig_face, cmap=plt.cm.gray)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### (b)

In [1179]:
tests_f = []
tests_f_hat = []
tests_y = []
for j in range(1, 3):
    for i in range(1, n+1):
        im = io.imread("../assets/assignment-5/problem1_files/faces_test/test{}/{}.pgm".format(j,i))
        h, w = np.shape(im)
        m = w * h
        f = vectorize_im(im, m)
        y = U_alpha.T @ (f - a)
        f_hat = a + U_alpha @ y

        tests_f.append(f)
        tests_f_hat.append(f_hat)
        tests_y.append(y)
        
tests_f = np.array(tests_f)
tests_f_hat = np.array(tests_f_hat)
tests_y = np.array(tests_y)

In [1180]:
fig, ax = plt.subplots(1, 2)
plt.subplots_adjust(wspace=0.1)

idx = np.random.randint(len(tests_f), size=1)[0]

test_ims = norm_im(tests_f[idx])
ax[0].set_axis_off()
ax[0].set_title(r"Original image")
ax[0].imshow(test_ims, cmap=plt.cm.gray)

test_im = norm_im(tests_f_hat[idx])

ax[1].set_axis_off()
ax[1].set_title(r"Reconstructed image")
ax[1].imshow(test_im, cmap=plt.cm.gray)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### (c)

In [1181]:
acc = 0
c_neighbors = [] 
for i in range(0, test_sets):
    for j in range(n):
        specimen = tests_y[i*n + j]
        match = sorted(training_y, key=lambda x: la.norm(specimen - x))[0] # sort and take first index.
        idx, _ = np.where(training_y == match) # suspicious
        if j == idx[0]:
            acc += 1 
        c_neighbors.append(idx[0])

In [1182]:
"accuracy = {:.2f} %".format(100 * acc / 2 / n)

'accuracy = 78.75 %'

In [1183]:
fig, ax = plt.subplots(1, 2)
plt.subplots_adjust(wspace=0.1)

idx = np.random.randint(len(tests_y), size=1)[0]

test_ims = norm_im(tests_f[idx])
ax[0].set_axis_off()
ax[0].set_title(r"Test image")
ax[0].imshow(test_ims, cmap=plt.cm.gray)

test_im = norm_im(training_f[c_neighbors[idx]])

ax[1].set_axis_off()
ax[1].set_title(r"Matched image")
ax[1].imshow(test_im, cmap=plt.cm.gray)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Q2

### (a)

In [1184]:
classes, labels, train_descr, test_descr = build_dataset("../assets/assignment-5/problem2_files/sift")

In [1185]:
all_train_descr = flatten(flatten(train_descr))
all_test_descr = flatten(flatten(test_descr))

In [1186]:
np.shape(all_test_descr)

(556114, 128)

In [1187]:
np.shape(all_train_descr)

(295000, 128)

In [1188]:
kmeans = KMeans(n_clusters=60, init='k-means++').fit(all_train_descr)

In [1189]:
kmeans.labels_

array([34, 50,  6, ..., 36, 27,  2], dtype=int32)

In [1190]:
kmeans.cluster_centers_

array([[0.07143946, 0.10625432, 0.07756905, ..., 0.01109407, 0.01175178,
        0.02017233],
       [0.07719146, 0.11278784, 0.06642741, ..., 0.01591448, 0.01133154,
        0.01641522],
       [0.03089918, 0.01728198, 0.02092396, ..., 0.03637616, 0.00640853,
        0.00822151],
       ...,
       [0.03894799, 0.03775369, 0.04140979, ..., 0.02012634, 0.01889076,
        0.02042107],
       [0.04542718, 0.02857494, 0.02926345, ..., 0.03114134, 0.02926558,
        0.03243295],
       [0.0779599 , 0.04077723, 0.02606042, ..., 0.04882429, 0.01131157,
        0.00976224]], dtype=float32)

### (b)

In [1191]:
train_bow_hist, train_labels = build_bow_hists(kmeans, train_descr)
test_bow_hist, test_labels  = build_bow_hists(kmeans, test_descr) 

In [1192]:
np.array(test_bow_hist).shape

(9,)

In [1262]:
np.array(train_bow_hist).shape

(9, 100, 60)

In [1343]:
clf = svm.SVC(C=30.0, decision_function_shape='ovo', probability=False, kernel='linear', cache_size=1000)

In [1344]:
clf.fit(flatten(train_bow_hist), flatten(train_labels))

SVC(C=30.0, break_ties=False, cache_size=1000, class_weight=None, coef0=0.0,
    decision_function_shape='ovo', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [1345]:
predicted = clf.predict(flatten(test_bow_hist))

In [1346]:
predicted

array([0, 0, 8, ..., 8, 8, 8])

In [1349]:
clf.score(flatten(test_bow_hist), flatten(test_labels))

0.6418879056047198

In [1348]:
print("Classification report for classifier %s:\n%s\n"
      % (clf, metrics.classification_report(flatten(test_labels), predicted)))
disp = metrics.plot_confusion_matrix(clf, flatten(test_bow_hist), flatten(test_labels))
disp.figure_.suptitle("Confusion Matrix")
print("Confusion matrix:\n%s" % disp.confusion_matrix)

Classification report for classifier SVC(C=30.0, break_ties=False, cache_size=1000, class_weight=None, coef0=0.0,
    decision_function_shape='ovo', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False):
              precision    recall  f1-score   support

           0       0.71      0.47      0.56       260
           1       0.77      0.91      0.84       228
           2       0.55      0.50      0.52       160
           3       0.49      0.55      0.52       110
           4       0.71      0.61      0.66       274
           5       0.62      0.67      0.64       115
           6       0.66      0.70      0.68       215
           7       0.53      0.56      0.54       192
           8       0.60      0.83      0.70       141

    accuracy                           0.64      1695
   macro avg       0.63      0.64      0.63      1695
weighted avg       0.65      0.64      0.64      1695




Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Confusion matrix:
[[121  16  44   5  32   0   3  29  10]
 [  0 208   0   0   8   0   5   3   4]
 [ 23   1  80   1   9   5   7  19  15]
 [  1   0   1  60   1  22   7   9   9]
 [ 23  32  11   0 167   1   5  17  18]
 [  0   0   2  25   2  77   4   1   4]
 [  0   9   2  12   4  12 151  17   8]
 [  2   3   4  10  11   4  41 107  10]
 [  0   1   2  10   1   4   6   0 117]]
